In [20]:
import sys

sys.path.append('../../src')
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource
from energiapy.components.process import Process
from energiapy.components.location import Location
from energiapy.components.transport import Transport
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.model.solve import solve


In [21]:
# ======================================================================================================================
# Initialize Case Study
# ======================================================================================================================
start_time = 0  # Start time of the analysis                        (t0)
end_time = 1000  # End time of the analysis                          (tf)
_time_intervals = 10  # Number of time intervals in a planning horizon    (L_chi)
_commodities = 1  # Number of commodities                             (rho)
_exec_scenarios = 4  # Number of execution scenarios                     (chi)

M = 1e7  # Big M

# Define temporal scales
scales = TemporalScale(discretization_list=[_exec_scenarios, _time_intervals])

In [22]:
# ======================================================================================================================
# Declare resources/commodities
# ======================================================================================================================

com1 = Resource(name='com1', cons_max=M, demand=True, sell=True, block={'imp': 1, 'urg': 1}, label='Commodity 1', revenue= 10000000.00, price= 7.50)

In [38]:
# ======================================================================================================================
# Declare processes/storage capacities
# ======================================================================================================================
store20 = Process(name='store20', storage=com1, store_max=20, prod_max=M, conversion={com1: -1, com1: 1}, capex=2000,
                  label="Storage capacity of 20 units")
store50 = Process(name='store50', storage=com1, store_max=50, prod_max=M, conversion={com1: -1, com1: 1}, capex=5000,
                  label="Storage capacity of 50 units")

In [39]:
# ======================================================================================================================
# Declare locations/warehouses
# ======================================================================================================================
loc1 = Location(name='loc1', processes={store50}, label="Location 1", scales=scales, demand_scale_level=1,
                capacity_scale_level=1, availability_scale_level=1)
loc2 = Location(name='loc2', processes={store20}, label="Location 2", scales=scales, demand_scale_level=1,
                capacity_scale_level=1, availability_scale_level=1)

In [40]:
# ======================================================================================================================
# Declare transport/trucks
# ======================================================================================================================
truck100 = Transport(name='truck100', resources=[com1], trans_max=100, label='Truck with maximum capacity of 100 units',
                     trans_cost=0.1)

In [41]:
transport_matrix = [
    [[], [truck100]],  # sink: location 1
    [[truck100], []],  # sink: location 2
]

In [42]:
distance_matrix = [
    [0, 10],
    [10, 0],
]

In [43]:
# ======================================================================================================================
# Declare network
# ======================================================================================================================
locset = {loc1, loc2}

sources = list(locset)
sinks = list(locset)

network = Network(name='Network', source_locations=sources, sink_locations=sinks, transport_matrix=transport_matrix,
                  distance_matrix=distance_matrix)

In [44]:
# ======================================================================================================================
# Declare scenario
# ======================================================================================================================
demand_dict = {i: {com1: 100} for i in locset if i == loc2}
scenario = Scenario(name='scenario', scales=scales, scheduling_scale_level=1, network_scale_level=0, expenditure_scale_level=0, purchase_scale_level=1,
                    demand_scale_level=1, network=network, demand=demand_dict, label='scenario')

In [45]:
problem = formulate(scenario=scenario, constraints={Constraints.COST, Constraints.TRANSPORT,
                                                    Constraints.RESOURCE_BALANCE, Constraints.PRODUCTION,
                                                    Constraints.INVENTORY},
                    objective=Objective.COST)

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint location capex
constraint location fopex
constraint location vopex
constraint location incidental
constraint network capex
constraint network fopex
constraint network vopex
constraint network incidental
constraint nameplate inventory
constraint storage max
constraint storage min
constraint production mode
constraint nameplate production
constraint production max
constraint production min
constraint inventory balance
constraint resource consumption
constraint resource purchase
constraint location production
constraint location discharge
constraint location consumption
constraint location purchase
constraint network production
constraint network discharge
constraint network consumption
constraint network purchase
constraint transport export
constraint transport import
constraint transport exp UB
constraint transport imp UB
constraint transport balance
constraint transport i

In [46]:
results = solve(scenario=scenario, instance=problem, solver='gurobi', name='LP')

Set parameter QCPDual to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2276 rows, 2093 columns and 5680 nonzeros
Model fingerprint: 0x60528635
Variable types: 2077 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 1e+07]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [47]:
results.model_summary()

SUMMARY:
            number of constraints: 2276
            number of variables: 2093
            number of binary variables: 16
            number of integer variables: 16
            number of continuous variables: 2061


In [48]:
demand_dict

{loc2: {com1: 100}}